In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import os
import time
from tqdm import tqdm, trange
import re

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
path_to_file = 'corpus.txt'
corpus = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [7]:
corpus = re.sub(r"[^a-zA-Z0-9 .,\[\]\(\)\n\']", "",corpus)

In [8]:
# The unique characters in the file
vocab = sorted(set(corpus))
print(f'{len(vocab)} unique characters')
print(f'Length of text: {len(corpus)} characters')

70 unique characters
Length of text: 3211598 characters


In [9]:
device = tf.device('CUDA') 

chars = tf.strings.unicode_split(vocab, input_encoding='UTF-8')


2023-07-29 18:27:16.293469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 18:27:16.293642: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 18:27:16.293738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [11]:
vocab

[' ',
 "'",
 '(',
 ')',
 ',',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [12]:
ids = ids_from_chars(chars)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(corpus, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"'JERRY Do you know what this is all about Do you know, why were here To be out, this is out...and ou"
Target: b'JERRY Do you know what this is all about Do you know, why were here To be out, this is out...and out'


2023-07-29 18:27:17.634116: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [3211598]
	 [[{{node Placeholder/_0}}]]


In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))


In [17]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [18]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [19]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [20]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

2023-07-29 18:27:17.696636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [3211598]
	 [[{{node Placeholder/_0}}]]
2023-07-29 18:27:17.696861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [3211598]
	 [[{{node Placeholder/_0}}]]


(64, 100, 71) # (batch_size, sequence_length, vocab_size)


2023-07-29 18:27:20.025420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-29 18:27:20.093809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [23]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ask you a question. How many people work at these big law officesMORTY Depends on the firm.JERRY Yea'

Next Char Predictions:
 b"[xvdXcGP[ fyHQs7wV.fF'oTE2T1(KNnVdZ3R412p0 0PVyBt4'iZs1wpvR55,F2uouZo.jA7Y(kMVC7[qHFwUHMLNAWikYlRP5'"


In [24]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 71)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.264075, shape=(), dtype=float32)


In [25]:
tf.exp(example_batch_mean_loss).numpy()

71.099106

In [26]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss=loss)
# Directory where the checkpoints will be saved
checkpoint_dir = './seinfeld_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [28]:
EPOCHS = 5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5


2023-07-29 18:27:20.264127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [3211598]
	 [[{{node Placeholder/_0}}]]
2023-07-29 18:27:20.264381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [3211598]
	 [[{{node Placeholder/_0}}]]
2023-07-29 18:27:20.403363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_d

496/496 [==============================] - 20s 37ms/step - loss: 2.1372
Epoch 2/5
496/496 [==============================] - 19s 37ms/step - loss: 1.4595
Epoch 3/5
496/496 [==============================] - 19s 37ms/step - loss: 1.3072
Epoch 4/5
496/496 [==============================] - 19s 37ms/step - loss: 1.2311
Epoch 5/5
496/496 [==============================] - 19s 37ms/step - loss: 1.1796


In [29]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)
    print(skip_ids)

  @tf.function
  def generate_one_step(self, inputs, states=None,mask=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()
    
    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    print(predicted_logits)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask
    print(predicted_logits)

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    print(predicted_ids)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)
    print(predicted_chars)
    print(states)
    # Return the characters and model state.
    return predicted_chars, states

In [30]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

tf.Tensor([[0]], shape=(1, 1), dtype=int64)


In [31]:
one_step_model.generate_one_step(tf.constant(['Jerry ']))

Tensor("my_model/dense/BiasAdd:0", shape=(1, None, 71), dtype=float32)
Tensor("add:0", shape=(1, 71), dtype=float32)
Tensor("Squeeze:0", shape=(1,), dtype=int64)
Tensor("string_lookup_1/None_Lookup/LookupTableFindV2:0", shape=(1,), dtype=string)
Tensor("my_model/gru/PartitionedCall:2", shape=(1, 1024), dtype=float32)


2023-07-29 18:28:54.982720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-29 18:28:54.983527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-29 18:28:54.984113: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b't'], dtype=object)>,
 <tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
 array([[ 0.0964728 , -0.53576314,  0.63929415, ...,  0.17748895,
         -0.9191218 ,  0.04077792]], dtype=float32)>)

In [32]:
start = time.time()
states = None
next_char = tf.constant(['JERRY:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

2023-07-29 18:28:55.191104: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-29 18:28:55.191899: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-29 18:28:55.192491: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Tensor("my_model/dense/BiasAdd:0", shape=(1, None, 71), dtype=float32)
Tensor("add:0", shape=(1, 71), dtype=float32)
Tensor("Squeeze:0", shape=(1,), dtype=int64)
Tensor("string_lookup_1/None_Lookup/LookupTableFindV2:0", shape=(1,), dtype=string)
Tensor("my_model/gru/PartitionedCall:2", shape=(1, 1024), dtype=float32)
JERRY: I wish I haven't food, turn you out therejarryuaring him and you have a carDRADE They have together 

________________________________________________________________________________

Run time: 0.34567832946777344


In [33]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [34]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [35]:
model.compile(optimizer = tf.keras.optimizers.legacy.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [36]:
model.fit(dataset, epochs=100)

Epoch 1/100


2023-07-29 18:28:55.978897: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-29 18:28:55.979813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-29 18:28:55.980453: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

496/496 [==============================] - 20s 36ms/step - loss: 2.1280
Epoch 2/100
496/496 [==============================] - 19s 36ms/step - loss: 1.4566
Epoch 3/100
496/496 [==============================] - 19s 36ms/step - loss: 1.3049
Epoch 4/100
496/496 [==============================] - 19s 36ms/step - loss: 1.2298
Epoch 5/100
496/496 [==============================] - 19s 36ms/step - loss: 1.1790
Epoch 6/100
496/496 [==============================] - 19s 36ms/step - loss: 1.1382
Epoch 7/100
496/496 [==============================] - 19s 36ms/step - loss: 1.1032
Epoch 8/100
496/496 [==============================] - 19s 36ms/step - loss: 1.0705
Epoch 9/100
496/496 [==============================] - 19s 36ms/step - loss: 1.0394
Epoch 10/100
496/496 [==============================] - 19s 36ms/step - loss: 1.0108
Epoch 11/100
496/496 [==============================] - 19s 36ms/step - loss: 0.9832
Epoch 12/100
496/496 [==============================] - 19s 36ms/step - loss: 0.9580
E

496/496 [==============================] - 19s 36ms/step - loss: 1.2055
Epoch 98/100
496/496 [==============================] - 19s 36ms/step - loss: 1.1953
Epoch 99/100
496/496 [==============================] - 19s 36ms/step - loss: 1.1865
Epoch 100/100
496/496 [==============================] - 19s 36ms/step - loss: 1.1779


In [37]:
start = time.time()
states = None
next_char = tf.constant(['JERRY:'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

JERRY:GE We still dount O'MR. II'd take the new Joe Mayo Paco.ELAINE Blooding Move good, the onl of the middle of small than mind. I'm sorry I'm so sign anymore, moving the ross. (exits)ELAINE You stood Can't turn youKRAMER (standing) Gimme a see, I'm heading to play gift. We're going to get a Mock's playing a bit of a mail.JERRY Are you excuse meMELISSA Anyway, you said you're the just has a terrible of mine. He couldn't belong for Mr.. You see what you make your celeble. Ok, Kramer, we could try to eat of the few thousandsSUE ELLEN MINUCE (indippiew) Gove. Oh, no. If only strange something or I got no plot for yourself any more finally going more eighty bucket. Too much stand. I'm hating you. Look at this. This is a person was... I want you back there and we were colfered somebody in the first plate, right um... (Jerry room.JERRY You know hawnsELAINE I banged it out tonous moutrs. Viso bullsCLICK I don't like the most of six morning. I Lorge is going on in there I'm gonna take a rise

In [38]:
one_step_model.generate_one_step(tf.constant(['JERRY:']))

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b' '], dtype=object)>,
 <tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
 array([[-0.60128987,  0.11349986, -0.16860789, ...,  0.17382304,
          0.6948192 ,  0.03868974]], dtype=float32)>)